In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from math import factorial
import numpy as np
from scipy import stats as st

In [3]:
visitas = pd.read_csv('datos/visits_log_us.csv')

In [4]:
pedidos = pd.read_csv('datos/orders_log_us.csv')

In [5]:
gastos = pd.read_csv('datos/costs_us.csv')

VISITAS 

In [6]:
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


Device    0.0
dtype: float64


In [8]:
visitas = visitas.isnull().mean() * 100
print(visitas)

Device       0.0
End Ts       0.0
Source Id    0.0
Start Ts     0.0
Uid          0.0
dtype: float64


In [9]:
visitas.duplicated().sum()

np.int64(4)

In [10]:
visitas = visitas.drop_duplicates()
print(visitas.duplicated().sum())
visitas.reset_index(drop=True)

0


0    0.0
dtype: float64